In [32]:
# # ================================================
# # Similar Product Search - Exercises 2 & 3
# # ================================================

# import pymongo
# from datasketch import MinHash, MinHashLSH
# from tqdm import tqdm
# import nltk
# from nltk.corpus import stopwords
# import pandas as pd
# import matplotlib.pyplot as plt

# # -----------------------------
# # ROUGE-L function
# # -----------------------------
# def rouge_l_f1(pred, ref):
#     pred_tokens = pred.lower().split()
#     ref_tokens = ref.lower().split()
#     n, m = len(pred_tokens), len(ref_tokens)
#     dp = [[0]*(m+1) for _ in range(n+1)]
#     for i in range(n):
#         for j in range(m):
#             if pred_tokens[i] == ref_tokens[j]:
#                 dp[i+1][j+1] = dp[i][j]+1
#             else:
#                 dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
#     lcs = dp[n][m]
#     if lcs == 0:
#         return 0.0
#     prec = lcs / n
#     rec = lcs / m
#     f1 = (2*prec*rec)/(prec+rec) if (prec+rec) > 0 else 0.0
#     return f1

# # -----------------------------
# # Stopwords
# # -----------------------------
# nltk.download('stopwords')
# STOP_WORDS = set(stopwords.words('english'))

# # -----------------------------
# # MongoDB Setup
# # -----------------------------
# MONGO_URI = "mongodb://localhost:27017"
# DB_NAME = "cs5600"
# client = pymongo.MongoClient(MONGO_URI)
# db = client[DB_NAME]
# product_col = db["products"]
# signature_col = db["productsignatures"]

# # Drop old signatures
# result = signature_col.delete_many({})
# print(f"✅ Dropped {result.deleted_count} old signature documents.")

# # -----------------------------
# # Helpers
# # -----------------------------
# def remove_stopwords(text):
#     tokens = text.lower().split()
#     return " ".join([t for t in tokens if t not in STOP_WORDS])

# def get_text(product, mode):
#     title = product.get("title", "") or ""
#     desc = " ".join(product.get("description", []) or [])
#     if mode == "pst":
#         return title.lower()
#     elif mode == "psd":
#         return remove_stopwords(desc)
#     else:
#         return remove_stopwords(title + " " + desc)

# def get_shingles(text, k, mode):
#     if mode == "pst":
#         text = text.replace(" ", "")
#         return set([text[i:i+k] for i in range(len(text)-k+1)]) if len(text) >= k else set([text])
#     else:
#         tokens = text.split()
#         return set([" ".join(tokens[i:i+k]) for i in range(len(tokens)-k+1)]) if len(tokens) >= k else set(tokens)

# def build_lsh(products, mode, shingle_size, num_perm, threshold):
#     # Optional: compute b,r
#     def calc_b_r(threshold, num_perm):
#         import math
#         best_diff = float('inf')
#         best_b, best_r = 1, num_perm
#         for r in range(1, num_perm+1):
#             b = num_perm // r
#             if b == 0:
#                 continue
#             approx_thr = 1 - (1 - threshold**r)**b
#             diff = abs(approx_thr - threshold)
#             if diff < best_diff:
#                 best_diff = diff
#                 best_b, best_r = b, r
#         return best_b, best_r
#     b, r = calc_b_r(threshold, num_perm)
#     print(f"Mode={mode}: Using b={b}, r={r} (threshold={threshold}, num_perm={num_perm})")

#     lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
#     sigs = {}
#     for p in products:
#         text = get_text(p, mode)
#         if not text.strip():
#             continue
#         shingles = get_shingles(text, shingle_size, mode)
#         m = MinHash(num_perm=num_perm)
#         for sh in shingles:
#             m.update(sh.encode("utf8"))
#         sigs[p["asin"]] = m
#         lsh.insert(p["asin"], m)
#     return lsh, sigs

# # -----------------------------
# # Precision@k + ROUGE-L evaluation
# # -----------------------------
# def precision_at_k_rouge(eval_products, mode, lsh, sigs, products, k=10):
#     asin_to_product = {p["asin"]: p for p in products if "asin" in p}
#     precisions, rouges = [], []

#     for p in eval_products:
#         asin = p["asin"]
#         if asin not in sigs:
#             continue
#         m = sigs[asin]
#         cands = lsh.query(m)
#         scored = [(c, m.jaccard(sigs[c])) for c in cands if c in sigs and c != asin]
#         top = [c for c, _ in sorted(scored, key=lambda x: -x[1])[:k]]
#         gt_asins = set(p.get("similar_asins", []))
#         if not gt_asins:
#             continue

#         hits = sum(1 for c in top if c in gt_asins)
#         precisions.append(hits / k)

#         rouge_scores = []
#         for c in top:
#             pred_title = asin_to_product.get(c, {}).get("title", "")
#             rouge_c = max(
#                 (rouge_l_f1(pred_title, asin_to_product.get(gt, {}).get("title", "")) 
#                  for gt in gt_asins if gt in asin_to_product),
#                 default=0.0
#             )
#             rouge_scores.append(rouge_c)
#         rouges.append(sum(rouge_scores)/k if rouge_scores else 0.0)

#     return {
#         "precision@k": sum(precisions)/len(precisions) if precisions else 0.0,
#         "rouge_l": sum(rouges)/len(rouges) if rouges else 0.0
#     }

# # -----------------------------
# # Hyperparameter tuning
# # -----------------------------
# def tune(products, eval_products, save_csv=False):
#     TOP_K = 10

#     param_grid = {
#         "shingle_size": [2, 3, 5, 7, 10],
#         "num_perm": [10, 20, 50, 100, 150],
#         "threshold": [0.2, 0.3, 0.4, 0.5]
#     }

#     grid_results = []
#     best_params = {}

#     for mode in ["pst", "psd", "pstd"]:
#         best_score = -1
#         best_config = None
#         print(f"\n--- Tuning {mode} ---")
#         for shingle_size in param_grid["shingle_size"]:
#             for num_perm in param_grid["num_perm"]:
#                 for thr in param_grid["threshold"]:
#                     lsh, sigs = build_lsh(eval_products, mode, shingle_size, num_perm, thr)
#                     if not sigs:
#                         continue
#                     res = precision_at_k_rouge(eval_products, mode, lsh, sigs, products, TOP_K)
#                     combined_score = 0.5*res["precision@k"] + 0.5*res["rouge_l"]
#                     grid_results.append({
#                         "mode": mode,
#                         "shingle_size": shingle_size,
#                         "num_perm": num_perm,
#                         "threshold": thr,
#                         "precision@10": res["precision@k"],
#                         "ROUGE-L": res["rouge_l"],
#                         "MAP@10": res["precision@k"]
#                     })
#                     if combined_score > best_score:
#                         best_score = combined_score
#                         best_config = {
#                             "shingle_size": shingle_size,
#                             "num_perm": num_perm,
#                             "threshold": thr,
#                             "MAP@10": res["precision@k"],
#                             "ROUGE-L": res["rouge_l"]
#                         }
#         best_params[mode] = best_config
#         print(f"\n✅ Best for {mode}: {best_config}")

#     if save_csv:
#         df_grid = pd.DataFrame(grid_results)
#         df_grid.to_csv("hyperparam_ex3.csv", index=False)
#         print("✅ Hyperparameter results saved as hyperparam_ex3.csv")
#         for mode in ["pst", "psd"]:
#             df_mode = df_grid[df_grid.mode==mode]
#             plt.figure(figsize=(8,5))
#             for thr in df_mode.threshold.unique():
#                 subset = df_mode[df_mode.threshold==thr]
#                 plt.plot(subset.num_perm, subset['MAP@10'], label=f"thr={thr}")
#             plt.title(f"MAP@10 vs num_perm for {mode}")
#             plt.xlabel("num_perm")
#             plt.ylabel("MAP@10")
#             plt.legend()
#             plt.savefig(f"plot_{mode}.png")
#             plt.close()
#         print("✅ Plots saved for Exercise 3")

#     return best_params

# # -----------------------------
# # Exercise 2: Precompute top-K similar products
# # -----------------------------
# def precompute(best_params, products, TOP_K=10):
#     for mode, params in best_params.items():
#         if not params:
#             continue
#         lsh, sigs = build_lsh(products, mode, params["shingle_size"], params["num_perm"], params["threshold"])
#         print(f"Storing {mode} results...")
#         for p in tqdm(products):
#             asin = p["asin"]
#             if asin not in sigs:
#                 continue
#             m = sigs[asin]
#             candidates = lsh.query(m)
#             scored = [(c, m.jaccard(sigs[c])) for c in candidates if c != asin]
#             top = [{"asin": c, "score": s} for c, s in sorted(scored, key=lambda x: -x[1])[:TOP_K]]
#             signature_col.update_one(
#                 {"asin": asin},
#                 {"$set": {f"similar.{mode}": top}},
#                 upsert=True
#             )

# # -----------------------------
# # Run pipeline
# # -----------------------------
# # Products for Exercise 2 (all with at least one similar)
# products_e2 = list(product_col.find({}, {"asin":1,"title":1,"description":1}))
# # All products for Exercise 3 + top-100 eval
# all_products = list(product_col.find({}, {"asin":1,"title":1,"description":1,"similar_asins":1}))
# eval_products = sorted([p for p in all_products if p.get("similar_asins")], key=lambda x: len(x["similar_asins"]), reverse=True)[:100]

# # --- Exercise 2: grid search + precompute ---
# best_params_e2 = tune(products_e2, products_e2)
# precompute(best_params_e2, products_e2)

# # --- Exercise 3: grid search + CSV/plots ---
# best_params_e3 = tune(all_products, eval_products, save_csv=True)

# print("✅ Done: Exercise 2 & 3 completed. Data stored in DB and CSV/plots generated.")




In [33]:
import pymongo
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
import pandas as pd
import matplotlib.pyplot as plt

# NLTK stopwords
nltk.download('stopwords')
STOP_WORDS = set(stopwords.words('english'))

# MongoDB Setup
MONGO_URI = "mongodb://localhost:27017"
DB_NAME = "cs5600"
client = pymongo.MongoClient(MONGO_URI)
db = client[DB_NAME]
product_col = db["products"]
signature_col = db["productsignatures"]

# Drop old signatures
result = signature_col.delete_many({})
print(f"✅ Dropped {result.deleted_count} old signature documents.")


[nltk_data] Downloading package stopwords to /home/suvedh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Dropped 2000 old signature documents.


In [34]:
# -----------------------------
# Text processing
# -----------------------------
def remove_stopwords(text):
    tokens = text.lower().split()
    return " ".join([t for t in tokens if t not in STOP_WORDS])

def get_text(product, mode):
    title = product.get("title", "") or ""
    desc = " ".join(product.get("description", []) or [])
    if mode == "pst":
        return title.lower()
    elif mode == "psd":
        return remove_stopwords(desc)
    else:
        return remove_stopwords(title + " " + desc)

def get_shingles(text, k, mode):
    if mode == "pst":
        text = text.replace(" ", "")
        return set([text[i:i+k] for i in range(len(text)-k+1)]) if len(text) >= k else set([text])
    else:
        tokens = text.split()
        return set([" ".join(tokens[i:i+k]) for i in range(len(tokens)-k+1)]) if len(tokens) >= k else set(tokens)


In [35]:
def build_lsh(products, mode, shingle_size, num_perm, threshold):
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    sigs = {}
    for p in products:
        text = get_text(p, mode)
        if not text.strip():
            continue
        shingles = get_shingles(text, shingle_size, mode)
        m = MinHash(num_perm=num_perm)
        for sh in shingles:
            m.update(sh.encode("utf8"))
        sigs[p["asin"]] = m
        lsh.insert(p["asin"], m)
    return lsh, sigs


In [36]:
def rouge_l_f1(pred, ref):
    pred_tokens = pred.lower().split()
    ref_tokens = ref.lower().split()
    n, m = len(pred_tokens), len(ref_tokens)
    dp = [[0]*(m+1) for _ in range(n+1)]
    for i in range(n):
        for j in range(m):
            if pred_tokens[i] == ref_tokens[j]:
                dp[i+1][j+1] = dp[i][j]+1
            else:
                dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
    lcs = dp[n][m]
    if lcs == 0:
        return 0.0
    prec = lcs / n
    rec = lcs / m
    f1 = (2*prec*rec)/(prec+rec) if (prec+rec) > 0 else 0.0
    return f1

def precision_at_k_rouge(eval_products, mode, lsh, sigs, products, k=10):
    asin_to_product = {p["asin"]: p for p in products if "asin" in p}
    precisions, rouges = [], []

    for p in eval_products:
        asin = p["asin"]
        if asin not in sigs:
            continue
        m = sigs[asin]
        cands = lsh.query(m)
        scored = [(c, m.jaccard(sigs[c])) for c in cands if c in sigs and c != asin]
        top = [c for c, _ in sorted(scored, key=lambda x: -x[1])[:k]]
        gt_asins = set(p.get("similar_asins", []))
        if not gt_asins:
            continue

        hits = sum(1 for c in top if c in gt_asins)
        precisions.append(hits / k)

        rouge_scores = []
        for c in top:
            pred_title = asin_to_product.get(c, {}).get("title", "")
            rouge_c = max(
                (rouge_l_f1(pred_title, asin_to_product.get(gt, {}).get("title", "")) 
                 for gt in gt_asins if gt in asin_to_product),
                default=0.0
            )
            rouge_scores.append(rouge_c)
        rouges.append(sum(rouge_scores)/k if rouge_scores else 0.0)

    return {
        "precision@k": sum(precisions)/len(precisions) if precisions else 0.0,
        "rouge_l": sum(rouges)/len(rouges) if rouges else 0.0
    }


In [ ]:
def tune(products, eval_products, save_csv=False):
    TOP_K = 10

    param_grid = {
        "shingle_size": [2, 3, 5, 7, 10],
        "num_perm": [10, 20, 50, 100, 150],
        "threshold": [0.2, 0.3, 0.4, 0.5]
    }

    grid_results = []
    best_params = {}
    for mode in ["pst", "psd", "pstd"]:
        best_score = -1
        best_config = None
        print(f"\n--- Tuning {mode} ---")
        for shingle_size in param_grid["shingle_size"]:
            for num_perm in param_grid["num_perm"]:
                for thr in param_grid["threshold"]:
                    lsh, sigs = build_lsh(eval_products, mode, shingle_size, num_perm, thr)
                    if not sigs:
                        continue
                    res = precision_at_k_rouge(eval_products, mode, lsh, sigs, products, TOP_K)
                    combined_score = 0.5*res["precision@k"] + 0.5*res["rouge_l"]
                    grid_results.append({
                        "mode": mode,
                        "shingle_size": shingle_size,
                        "num_perm": num_perm,
                        "threshold": thr,
                        "precision@10": res["precision@k"],
                        "ROUGE-L": res["rouge_l"],
                        "MAP@10": res["precision@k"]
                    })
                    if combined_score > best_score:
                        best_score = combined_score
                        best_config = {
                            "shingle_size": shingle_size,
                            "num_perm": num_perm,
                            "threshold": thr,
                            "MAP@10": res["precision@k"],
                            "ROUGE-L": res["rouge_l"]
                        }
        best_params[mode] = best_config
        print(f"\n✅ Best for {mode}: {best_config}")

    if save_csv:
        df_grid = pd.DataFrame(grid_results)
        if 'mode' not in df_grid.columns:
            print("❌ 'mode' column missing in grid results. Skipping plotting.")
        else:
            df_grid.to_csv("hyperparam_ex3.csv", index=False)
            print("✅ Hyperparameter results saved as hyperparam_ex3.csv")

            for mode in ["pst", "psd"]:
                if mode not in df_grid['mode'].values:
                    continue
                df_mode = df_grid[df_grid['mode'] == mode]
                if df_mode.empty:
                    continue
                plt.figure(figsize=(8,5))
                for thr in df_mode.threshold.unique():
                    subset = df_mode[df_mode.threshold==thr]
                    plt.plot(subset.num_perm, subset['MAP@10'], label=f"thr={thr}")
                plt.title(f"MAP@10 vs num_perm for {mode}")
                plt.xlabel("num_perm")
                plt.ylabel("MAP@10")
                plt.legend()
                plt.savefig(f"plot_{mode}.png")
                plt.close()
            print("✅ Plots saved for Exercise 3")

    return best_params


In [38]:
def precompute(best_params, products, TOP_K=10):
    for mode, params in best_params.items():
        if not params:
            continue
        lsh, sigs = build_lsh(products, mode, params["shingle_size"], params["num_perm"], params["threshold"])
        print(f"Storing {mode} results...")
        for p in tqdm(products):
            asin = p["asin"]
            if asin not in sigs:
                continue
            m = sigs[asin]
            candidates = lsh.query(m)
            scored = [(c, m.jaccard(sigs[c])) for c in candidates if c != asin]
            top = [{"asin": c, "score": s} for c, s in sorted(scored, key=lambda x: -x[1])[:TOP_K]]
            signature_col.update_one(
                {"asin": asin},
                {"$set": {f"similar.{mode}": top}},
                upsert=True
            )


In [39]:
# --- Top 2000 products with similar_asins for Exercise 2 ---
all_products = list(product_col.find({}, {"asin":1,"title":1,"description":1,"similar_asins":1}))
products_e2 = sorted([p for p in all_products if p.get("similar_asins")], key=lambda x: len(x["similar_asins"]), reverse=True)[:2000]

# --- Exercise 2: grid search + precompute ---
best_params_e2 = tune(products_e2, products_e2)
precompute(best_params_e2, all_products)  # apply best model to all products

# --- Exercise 3: all products + top-100 eval ---
eval_products = sorted([p for p in all_products if p.get("similar_asins")], key=lambda x: len(x["similar_asins"]), reverse=True)[:100]
best_params_e3 = tune(all_products, eval_products, save_csv=True)

print("✅ Done: Exercise 2 & 3 completed. Data stored in DB and CSV/plots generated.")



--- Tuning pst ---

✅ Best for pst: {'shingle_size': 2, 'num_perm': 150, 'threshold': 0.2, 'MAP@10': 0.24480000000000002, 'ROUGE-L': 0.49931530712684075}

--- Tuning psd ---

✅ Best for psd: {'shingle_size': 2, 'num_perm': 20, 'threshold': 0.2, 'MAP@10': 0.11108786610878661, 'ROUGE-L': 0.28361494466939013}

--- Tuning pstd ---

✅ Best for pstd: {'shingle_size': 2, 'num_perm': 20, 'threshold': 0.2, 'MAP@10': 0.12110000000000001, 'ROUGE-L': 0.3000335170971946}
Storing pst results...


100%|██████████| 30239/30239 [11:31<00:00, 43.72it/s] 


Storing psd results...


100%|██████████| 30239/30239 [00:50<00:00, 603.39it/s] 


Storing pstd results...


100%|██████████| 30239/30239 [01:01<00:00, 492.00it/s] 



--- Tuning pst ---

✅ Best for pst: {'shingle_size': 3, 'num_perm': 20, 'threshold': 0.2, 'MAP@10': 0.171, 'ROUGE-L': 0.3445238091460831}

--- Tuning psd ---

✅ Best for psd: {'shingle_size': 2, 'num_perm': 20, 'threshold': 0.2, 'MAP@10': 0.06875, 'ROUGE-L': 0.12887353357004172}

--- Tuning pstd ---

✅ Best for pstd: {'shingle_size': 2, 'num_perm': 10, 'threshold': 0.2, 'MAP@10': 0.066, 'ROUGE-L': 0.1214330979350905}
✅ Hyperparameter results saved as hyperparam_ex3.csv
✅ Plots saved for Exercise 3
✅ Done: Exercise 2 & 3 completed. Data stored in DB and CSV/plots generated.
